In [1]:
using JuMP
using Gurobi
using DataFrames
using CSV
using LinearAlgebra
using Plots
using Statistics

In [2]:
A = zeros(18,15+11+55+18);
C = zeros(15+11+55+18)

99-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [3]:
#Shifts of 4h
for j=1:14
    A[j:j+3,j].+=1
    C[j] = 25*4
end

#Shifts of 8h
for j=1:10
    A[j:j+7,j+15].+=1
    C[j+15] = 20*8
end

#Shifts of 1h
for j=1:18
    A[j,j+15+55+11]+=1
    C[j+15+55+11] = 40
end

#Overtime Shifts
n = 0
for k=9:18
    for j=1:(18-k+1)
        A[j:j+k-1,j+15+11+n].+=1
        if k<11
            C[j+15+11+n] = 20*8+30*(k-8)
        else
            C[j+15+11+n] = 20*8+30*2+35*(k-10)
        end
    end
    n+=(18-k+1)
end

In [4]:
D1 = convert(Matrix,CSV.File("HW2_demand.csv"; header=true) |> DataFrame!)[:,2:73];
D = D1[:,1:4:72]

35×18 Array{Any,2}:
  4  29  72   54   80  26  15  40   87  47  31  34   35  77  36  51  27   4
  7  21  50  118   62  54  27  77   45  42  46  73   53  86  45  37  26  19
 24   4  35   51   93  36  15  67   84  37  45  54   51  78  71  53  26  13
 12  19  38   76   47  21   6  62   48  44  38  19   43  55  70  40  39  13
 11   6  46   70  109  34  20  51   62  59  15  46   49  51  46  40  44  23
  8  15  54  109   53  35  28  51   69  66  32  48   52  60  38  40  11   6
  7  14  33   60   41  23  24  46   68  21  56  53   40  91  63  35  29  18
 11   9  64   29   53  44  23  40   52  32  43  64   43  57  28  47  20  16
  4  11  51  103   39  30  26  48   86  54  29  59   68  66  69  80  41  23
  4  14  49   66   41  29  29  55   43  66  22  48   76  89  44  63  40  12
  4   4  44   62   65  34  23  77  125  33  30  46   58  89  56  46  41  12
 17  11  39   70   61  36  17  63   97  48  18  55   44  60  70  39  37  14
  4   4  52   78   73  24  13  31   76  30  18  75   58  67  45  53 

In [5]:
A

18×99 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [6]:
C

99-element Array{Float64,1}:
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
   ⋮
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0
  40.0

In [9]:
"Build shift allocation model 1"
function allocation_1(D, A, C)
    # extract problem dimensions from distance matrix and verify coherence of input data
    T,J  = size(A)
    M = size(D)[1]
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 30)
    # VARIABLES
    # How many of each shift to use
    @variable(model, x[1:M,1:J] >=0, Int)
    
    #Resource allocation constraint
    @constraint(model, enough_employees[t=1:T, d=1:M],
            sum(A[t, j]*x[d,j]  for j = 1:J) >= D[d, t])
    
    # OBJECTIVE
    @objective(
        model, Min, sum(C[j] * x[d, j] for d=1:M, j=1:J)
    )
    
    optimize!(model)
    
    return model, objective_value(model), x
end

allocation_1

In [23]:
m1, o1, x1 = allocation_1(D, A, C)
X1 = value.(x1)
print("Objective value: ", o1)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 630 rows, 3465 columns and 28490 nonzeros
Model fingerprint: 0x555787a1
Variable types: 0 continuous, 3465 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+02]
Found heuristic solution: objective 1102160.0000
Presolve removed 612 rows and 3368 columns
Presolve time: 0.04s
Presolved: 18 rows, 97 columns, 814 nonzeros
Found heuristic solution: objective 700355.00000
Variable types: 0 continuous, 97 integer (0 binary)

Root relaxation: objective 6.856550e+05, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0    

In [24]:
X1

35×99 Array{Float64,2}:
 -0.0  14.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  26.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  13.0  -0.0  -0.0  -0.0     -0.0  26.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   3.0  -0.0  -0.0  -0.0     -0.0   7.0  -0.0  -0.0  -0.0   7.0
 -0.0   3.0  -0.0  -0.0  -0.0  -0.0      4.0  -0.0   5.0  -0.0  -0.0  -0.0
 -0.0   6.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0   7.0  -0.0
  1.0  18.0   1.0  -0.0  -0.0  -0.0  …  -0.0   3.0  -0.0  -0.0  -0.0  -0.0
  1.0  17.0  -0.0   6.0  -0.0  -0.0     -0.0  27.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   8.0  -0.0  -0.0  -0.0     -0.0  10.0  -0.0  -0.0  -0.0   0.0
  4.0   9.0   1.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  11.0  -0.0  -0.0
  4.0  10.0  -0.0  -0.0  -0.0  -0.0      1.0  14.0  -0.0  -0.0   4.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  10.0  -0.0  -0.0   7.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0   7.0  -0.0  -0.0  -0.0
 -0.0  21.0  -0.0  -0.0  -0.0  -0.0     -0.0   5.0  -0.0  -0.0  -0.0  -0.0
 

In [31]:
"Build shelter location model 1"
function allocation_2(D, A, C)
    # extract problem dimensions from distance matrix and verify coherence of input data
    T,J  = size(A)
    M = size(D)[1]
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 300)
    # VARIABLES
    # How many of each shift to use
    @variable(model, x[1:M,1:J] >=0, Int)
    # Match employees with shifts
    @variable(model, y[1:M, 1:90, 1:81], Bin)
    
    #Resource allocation constraint
    @constraint(model, enough_shifts[t=1:T, d=1:M],
            sum(A[t, j]*x[d,j]  for j = 1:J) >= D[d, t])
    
    #No more than max employees available
    @constraint(model, limit_fulltime[d=1:M],
            sum(x[d,j]  for j = 16:81) <= 60)
    @constraint(model, limit_parttime[d=1:M],
            sum(x[d,j]  for j = 1:15) <= 30)
    
    @constraint(model, no_more_two_days[d=1:33, k=1:90],
            sum(y[d,k,j]+y[d+1,k,j]+y[d+2,k,j] for j=1:81) <=2)
    
    @constraint(model, meet_shift_employees[d=1:M, j=1:81],
            sum(y[d,k,j]  for k = 1:90) >= x[d,j])
    
    # OBJECTIVE
    @objective(
        model, Min, sum(C[j] * x[d, j] for d=1:M, j=1:J)
    )
    
    optimize!(model)
    
    return model, objective_value(model), x, y
end

allocation_2

In [45]:
m2, o2, x2, y2 = allocation_2(D, A, C)
print("Objective value: ", o2)
X2 = value.(x2)
#Y = value.(y2)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6505 rows, 258615 columns and 1011020 nonzeros
Model fingerprint: 0x1ea6b2a6
Variable types: 0 continuous, 258615 integer (255150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 1102160.0000
Presolve removed 70 rows and 6370 columns
Presolve time: 0.97s
Presolved: 6435 rows, 252245 columns, 986760 nonzeros
Variable types: 0 continuous, 252245 integer (248850 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   18539    7.2842750e+05   3.939993e+04   0.000000e+00      5s
   27849    7.4344000e+05   3.199861e+04   0.000000e+00     10s
   36020    7.5166500e+05   1.419750e+

35×99 Array{Float64,2}:
 0.0   0.0  -0.0  -0.0  -0.0  -0.0  …   1.0  43.0   2.0  17.0  -0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      8.0  41.0   0.0  -0.0  -0.0  -0.0
 0.0   0.0  -0.0   0.0   0.0   0.0      6.0  33.0  26.0   8.0  -0.0  -0.0
 0.0  -0.0  -0.0  -0.0  -0.0  -0.0      4.0  16.0  31.0   1.0  -0.0  -0.0
 0.0   0.0  -0.0   0.0   0.0   0.0      9.0  11.0   6.0  -0.0   4.0  -0.0
 0.0  -0.0   0.0   0.0   0.0   0.0  …  14.0  22.0   0.0   2.0  -0.0  -0.0
 0.0  -0.0   0.0   0.0   0.0   0.0     -0.0  51.0  23.0  -0.0  -0.0  -0.0
 0.0   0.0   1.0   0.0   0.0   0.0      0.0  14.0  -0.0  19.0  -0.0  -0.0
 0.0  -0.0  -0.0   0.0   0.0   0.0     22.0  20.0  23.0  34.0  -0.0  -0.0
 0.0  -0.0   0.0   0.0   0.0   0.0     34.0  47.0   2.0  21.0  -0.0  -0.0
 0.0  -0.0   0.0   0.0   0.0   0.0  …  12.0  43.0  10.0   0.0  -0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      1.0  17.0  27.0  -0.0  -0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0     13.0  22.0  -0.0   8.0  -0.0  -0.0
 ⋮            